In [1]:
# основные библиотеки
import os
import random

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from pathlib import Path

pd.set_option('display.max_columns', 500)

from tqdm.notebook import tqdm
from IPython.display import clear_output

# визуализация
sns.set(style='darkgrid', palette='deep')
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = 8, 5
plt.rcParams['savefig.format'] = 'pdf'

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Read last raw data

In [2]:
cwd = Path.cwd().resolve()

In [3]:
base = cwd / "processed/final"

In [12]:
guarantees_latest = pd.read_csv(base / 'guarantees_latest.csv', sep=',', dtype={'id': "Int64"})
attributes_latest = pd.read_csv(base / 'attributes_latest.csv', sep=',')
files_latest = pd.read_csv(base / 'files_latest.csv', sep=',')

/var/folders/t6/500cgxq11vj9kz7r0r1lbw440000gn/T/ipykernel_48543/3402814732.py:1: DtypeWarning: Columns (0: error) have mixed types. Specify dtype option on import or set low_memory=False.
  guarantees_latest = pd.read_csv(base / 'guarantees_latest.csv', sep=',', dtype={'id': "Int64"})


In [6]:
files_latest[files_latest['id'] == 1962714]

,id,file_index,stored_filename,stored_path,original_filename,download_url,mime_type,download_status,sha256,document_index,document_number,page_count,run_id,file_exists
29364,1962714,1,1962714_1.pdf,/Users/home/Work/10-edu/data-science/thesis/co...,Бланк_гар_ИвБизнес_0230.pdf,https://zakupki.gov.ru/44fz/filestore/public/1...,application/pdf,DOWNLOADED,279142cb12f42503b9050fab3e0fcaa97155f37fa9c242...,1.0,02L1690300629018001001,2.0,34.0,True


In [7]:
attributes_latest[attributes_latest['id'] == 1962714].head()

,id,section,field_name,field_value,document_index,document_number,run_id
1235001,1962714,Сводная информация (верхний блок),Статус,Размещено,NaN,NaN,34.0
1235002,1962714,Сводная информация (верхний блок),Банк-гарант,"ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СБЕРБАНК РОССИИ""",NaN,NaN,34.0
1235003,1962714,Сводная информация (верхний блок),ИНН,7707083893,NaN,NaN,34.0
1235004,1962714,Сводная информация (верхний блок),КПП,773601001,NaN,NaN,34.0
1235005,1962714,Сводная информация (верхний блок),Номер извещения об осуществлении закупки,0136100008718000125,NaN,NaN,34.0


In [13]:
guarantees_latest[guarantees_latest['id'] == 1962714]

,id,status,general_url,documents_url,fetched_at,warnings,error,run_id
273248,1962714,OK,https://zakupki.gov.ru/epz/bankguarantee/guara...,https://zakupki.gov.ru/epz/bankguarantee/guara...,2026-02-06T07:30:16.976796+00:00,[],NaN,34.0


## Read datamart

<!-- meta -->
| id | run_id | 
<!-- bank -->
bank_inn | bank_name (Сокращенное название) | 
<!-- lg -->
issue_date | start_date | end_date | sum | currency | 
<!-- pcpl -->
pcpl_inn | pcpl_name (coalesce из нескольких полей, преимущественно короткое название) | pcpl_region | pcpl_city | pcpl_type
<!-- bene -->
bene_inn | bene_name (coalesce из нескольких полей, преимущественно короткое название) | pcpl_region | pcpl_city | pcpl_type |
<!-- procurement -->
ikz | coverage_type
<!-- other meta -->
guarantee_number | published_time | redaction_type | stored_filename | stored_path | sha256

In [91]:
df = pd.read_csv(
    base / 'wide_analytical_latest.csv', sep=',', 
    dtype={
        "pcpl_inn": str, "bene_inn": str, "bank_inn": str,
        # "run_id": int
        }
    )

In [92]:
df['run_id'] = df['run_id'].fillna(0).astype(int)
df['end_date'] = pd.to_datetime(df['end_date'])
df['issue_date'] = pd.to_datetime(df['issue_date'])
df['start_date'] = pd.to_datetime(df['start_date'])

In [36]:
from datetime import datetime

# date_string = '2018-06-29 20:50:00+03:00'
# format_string = '%Y-%m-%d %H:%M:%S%z'

# datetime_object = datetime.strptime(date_string, format_string)

# print(datetime_object)
# print(type(datetime_object))


In [37]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 260320 entries, 0 to 260319
Data columns (total 31 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   id                   260320 non-null  int64         
 1   run_id               260320 non-null  int64         
 2   bank_inn             260311 non-null  str           
 3   bank_name            260311 non-null  str           
 4   pcpl_inn             260306 non-null  str           
 5   pcpl_name            233340 non-null  str           
 6   pcpl_region          250116 non-null  str           
 7   pcpl_city            243401 non-null  str           
 8   pcpl_type            260311 non-null  str           
 9   bene_inn             260308 non-null  str           
 10  bene_name            260310 non-null  str           
 11  bene_region          252033 non-null  str           
 12  bene_city            251972 non-null  str           
 13  bene_type            2603

In [38]:
df[df['id'] == 1962714]

,id,run_id,bank_inn,bank_name,pcpl_inn,pcpl_name,pcpl_region,pcpl_city,pcpl_type,bene_inn,bene_name,bene_region,bene_city,bene_type,issue_date,start_date,end_date,sum_summary,sum_lower,currency_from_label,ikz,coverage_type,guarantee_number,published_time,redaction_type,stored_filename,stored_path,sha256,currency_symbol,currency_from_value,sum
260312,1962714,34,7707083893,ПАО СБЕРБАНК,3711024893,"ООО "" ИВБИЗНЕСАКТИВ""",Ивановская область,с Подвязновский,Юридическое лицо РФ,6903006290,УФСИН РОССИИ ПО ТВЕРСКОЙ ОБЛАСТИ,Тверская область,г Тверь,Федеральное государственное казенное учреждени...,2018-06-29,2018-06-29,2018-12-30,"267 059,25₽","267 059,25 (ОКВ 643)","Денежная сумма, ₽",181690300629069500100100020250000223,Обеспечение исполнения контракта,02L16903006290180010,2018-06-29 20:50:00+03:00,Действующая,1962714_1.pdf,/Users/home/Work/10-edu/data-science/thesis/co...,279142cb12f42503b9050fab3e0fcaa97155f37fa9c242...,₽,643.0,267059.25


In [84]:
# df[df['pcpl_region'] != 'г. Москва']

In [85]:
# df.sample(19)[['id', 'published_time']]

## Create final dataset

In [20]:
df[df['id'] == 1962714]

,id,run_id,bank_inn,bank_name,pcpl_inn,pcpl_name,pcpl_region,pcpl_city,pcpl_type,bene_inn,bene_name,bene_region,bene_city,bene_type,issue_date,start_date,end_date,sum_summary,sum_lower,currency_from_label,ikz,coverage_type,guarantee_number,published_time,redaction_type,stored_filename,stored_path,sha256,currency_symbol,currency_from_value,sum
30389,1962714,34,7707083893,ПАО СБЕРБАНК,3711024893,"ООО "" ИВБИЗНЕСАКТИВ""",Ивановская область,с Подвязновский,Юридическое лицо РФ,6903006290,УФСИН РОССИИ ПО ТВЕРСКОЙ ОБЛАСТИ,Тверская область,г Тверь,Федеральное государственное казенное учреждени...,2018-06-29,2018-06-29,2018-12-30,"267 059,25₽","267 059,25 (ОКВ 643)","Денежная сумма, ₽",181690300629069500100100020250000223,Обеспечение исполнения контракта,02L16903006290180010,2018-06-29 20:50:00+03:00,Действующая,1962714_1.pdf,/Users/home/Work/10-edu/data-science/thesis/co...,279142cb12f42503b9050fab3e0fcaa97155f37fa9c242...,₽,643.0,267059.25


In [43]:
# df[(~df['sum'].isna()) & (df['sum_summary'].isna())][['sum_lower', 'sum']]

In [54]:
df['stored_filename'].value_counts()

stored_filename
6_1.pdf          1
11_1.pdf         1
16_1.jpg         1
17_1.jpg         1
22_1.pdf         1
                ..
1962717_1.pdf    1
1962718_1.pdf    1
1962719_1.pdf    1
1962720_1.pdf    1
1962721_1.pdf    1
Name: count, Length: 260320, dtype: int64

In [56]:
df['currency'] = None 
df.loc[df['currency_from_value'] == 643, 'currency'] = 'RUB'

In [63]:
df = df[
    (df['pcpl_type'] == 'Юридическое лицо РФ') 
    # & (df_sub['bank_inn'])
]

In [86]:
df_sub = df[['id', 'bank_inn', 'bank_name', 'pcpl_inn',
       'bene_inn', 
       'issue_date', 'start_date',
       'end_date', 
       'sum', 'currency',
       'ikz',
       'stored_filename', 'stored_path']].copy()
# df_sub

In [89]:
# df_sub[df_sub['currency'].isna()]

In [75]:
res = df_sub.dropna().copy()

In [66]:
df_sub.info()

<class 'pandas.DataFrame'>
Index: 233333 entries, 0 to 260319
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   id               233333 non-null  int64         
 1   bank_inn         233333 non-null  str           
 2   bank_name        233333 non-null  str           
 3   pcpl_inn         233333 non-null  str           
 4   bene_inn         233330 non-null  str           
 5   issue_date       228569 non-null  datetime64[us]
 6   start_date       222465 non-null  datetime64[us]
 7   end_date         233333 non-null  datetime64[us]
 8   sum              233333 non-null  float64       
 9   currency         233332 non-null  object        
 10  ikz              233333 non-null  str           
 11  stored_filename  233333 non-null  str           
 12  stored_path      233333 non-null  str           
dtypes: datetime64[us](3), float64(1), int64(1), object(1), str(7)
memory usage: 69.9+ MB


In [76]:
res.info()

<class 'pandas.DataFrame'>
Index: 218412 entries, 479 to 260319
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   id               218412 non-null  int64         
 1   bank_inn         218412 non-null  str           
 2   bank_name        218412 non-null  str           
 3   pcpl_inn         218412 non-null  str           
 4   bene_inn         218412 non-null  str           
 5   issue_date       218412 non-null  datetime64[us]
 6   start_date       218412 non-null  datetime64[us]
 7   end_date         218412 non-null  datetime64[us]
 8   sum              218412 non-null  float64       
 9   currency         218412 non-null  object        
 10  ikz              218412 non-null  str           
 11  stored_filename  218412 non-null  str           
 12  stored_path      218412 non-null  str           
dtypes: datetime64[us](3), float64(1), int64(1), object(1), str(7)
memory usage: 65.5+ MB


In [77]:
res

,id,bank_inn,bank_name,pcpl_inn,bene_inn,issue_date,start_date,end_date,sum,currency,ikz,stored_filename,stored_path
479,1163,7707083893,"ОАО ""Сбербанк России""",4205241533,4212005791,2015-01-29,2015-01-29,2016-03-01,52351.13,RUB,,1163_1.pdf,/Users/home/Work/10-edu/data-science/thesis/co...
8280,53015,7707083893,"ОАО ""Сбербанк России""",2726000036,2703007684,2014-07-02,2014-07-02,2015-01-30,182690.84,RUB,,53015_1.TIF,/Users/home/Work/10-edu/data-science/thesis/co...
8281,55183,7707083893,"ОАО ""Сбербанк России""",7718538045,6905044950,2014-07-03,2014-07-03,2015-01-31,2094844.98,RUB,,55183_2.tiff,/Users/home/Work/10-edu/data-science/thesis/co...
8282,75918,7734202860,"ОАО ""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК""",7703784305,7453019764,2014-07-30,2014-07-30,2015-01-31,421000.00,RUB,,75918_1.pdf,/Users/home/Work/10-edu/data-science/thesis/co...
8283,96689,7729003482,"АКБ ""Держава"" ОАО",4826013297,4826008716,2014-08-27,2014-08-27,2015-01-01,1311317.00,RUB,,96689_1.pdf,/Users/home/Work/10-edu/data-science/thesis/co...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
260314,1962716,7707083893,ПАО СБЕРБАНК,4704043671,4704063710,2018-06-29,2018-06-29,2018-10-31,400000.00,RUB,,1962716_1.pdf,/Users/home/Work/10-edu/data-science/thesis/co...
260315,1962717,7707083893,ПАО СБЕРБАНК,2222805521,7902525986,2018-06-29,2018-06-29,2018-08-26,75000.00,RUB,,1962717_1.pdf,/Users/home/Work/10-edu/data-science/thesis/co...
260317,1962719,7707083893,ПАО СБЕРБАНК,3702148846,7815022288,2018-06-29,2018-06-29,2018-08-29,22400.49,RUB,,1962719_1.pdf,/Users/home/Work/10-edu/data-science/thesis/co...
260318,1962720,2801015394,"ПАО КБ ""ВОСТОЧНЫЙ""",7713327280,7734115270,2018-06-28,2018-06-29,2019-04-25,645774.00,RUB,,1962720_1.pdf,/Users/home/Work/10-edu/data-science/thesis/co...


In [79]:
(res == '').sum()

id                      0
bank_inn                0
bank_name               0
pcpl_inn                0
bene_inn                0
issue_date              0
start_date              0
end_date                0
sum                     0
currency                0
ikz                127728
stored_filename         0
stored_path             0
dtype: int64

In [83]:
res.to_csv(base / "dataset.csv", index=False)
print("Saved CSVs to", base)

Saved CSVs to /Users/home/Work/10-edu/data-science/thesis/code/masters-thesis-dev/data/processed/final


In [90]:
base / 'wide_analytical_latest.csv'

PosixPath('/Users/home/Work/10-edu/data-science/thesis/code/masters-thesis-dev/data/processed/final/wide_analytical_latest.csv')

### Changes example

In [70]:
files_latest[files_latest['id'] == 1961303]

,id,file_index,stored_filename,stored_path,original_filename,download_url,mime_type,download_status,sha256,document_index,document_number,page_count,run_id,file_exists
20581,1961303,1,1961303_1.pdf,/Users/home/Work/10-edu/data-science/thesis/co...,бг_333-35785.pdf,https://zakupki.gov.ru/44fz/filestore/public/1...,application/pdf,DOWNLOADED,d4bba090cb20aaa1c60585913ca3a524f1636613c2bc95...,1.0,06D2121500107718000606,2.0,34.0,True
20582,1961303,2,1961303_2.pdf,/Users/home/Work/10-edu/data-science/thesis/co...,Изменения к БГ 333-35785.pdf,https://zakupki.gov.ru/44fz/filestore/public/1...,application/pdf,DOWNLOADED,c9a70985c80c4ab9cd4de4b4a555587066af2c3885ab00...,1.0,06D2121500107718000606,1.0,34.0,True
20583,1961303,3,1961303_3.pdf,/Users/home/Work/10-edu/data-science/thesis/co...,бг_333-35785.pdf,https://zakupki.gov.ru/44fz/filestore/public/1...,application/pdf,DOWNLOADED,d4bba090cb20aaa1c60585913ca3a524f1636613c2bc95...,2.0,06D2121500107718000603,2.0,34.0,True
20584,1961303,4,1961303_4.pdf,/Users/home/Work/10-edu/data-science/thesis/co...,Изменения к БГ 333-35785.pdf,https://zakupki.gov.ru/44fz/filestore/public/1...,application/pdf,DOWNLOADED,c9a70985c80c4ab9cd4de4b4a555587066af2c3885ab00...,2.0,06D2121500107718000603,1.0,34.0,True
20585,1961303,5,1961303_5.pdf,/Users/home/Work/10-edu/data-science/thesis/co...,бг_333-35785.pdf,https://zakupki.gov.ru/44fz/filestore/public/1...,application/pdf,DOWNLOADED,d4bba090cb20aaa1c60585913ca3a524f1636613c2bc95...,3.0,06D2121500107718000601,2.0,34.0,True


In [73]:
attributes_latest[(attributes_latest['id'] == 1961303) & (attributes_latest['section'] == 'Документы: Информация о банковской гарантии')]

,id,section,field_name,field_value,document_index,document_number,run_id
824996,1961303,Документы: Информация о банковской гарантии,Номер банковской гарантии,06D2121500107718000606,1.0,06D2121500107718000606,34.0
824997,1961303,Документы: Информация о банковской гарантии,Размещено,29.06.2018 15:20 (МСК),1.0,06D2121500107718000606,34.0
824998,1961303,Документы: Информация о банковской гарантии,Редакция,Действующая,1.0,06D2121500107718000606,34.0
824999,1961303,Документы: Информация о банковской гарантии,Тип изменений,Внесение изменений в информацию о банковской г...,1.0,06D2121500107718000606,34.0
825000,1961303,Документы: Информация о банковской гарантии,Описание изменений,.,1.0,06D2121500107718000606,34.0
825001,1961303,Документы: Информация о банковской гарантии,Дата внесения изменений,29.06.2018,1.0,06D2121500107718000606,34.0
825002,1961303,Документы: Информация о банковской гарантии,Номер банковской гарантии,06D2121500107718000603,2.0,06D2121500107718000603,34.0
825003,1961303,Документы: Информация о банковской гарантии,Размещено,29.06.2018 15:18 (МСК),2.0,06D2121500107718000603,34.0
825004,1961303,Документы: Информация о банковской гарантии,Редакция,Недействующая,2.0,06D2121500107718000603,34.0
825005,1961303,Документы: Информация о банковской гарантии,Тип изменений,Внесение изменений в условия банковской гарантии,2.0,06D2121500107718000603,34.0
